In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.data import make_dataset
from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANLIZAR = 112103  # Este es el Torax


## Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s

In [3]:
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
)

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(
    df_nacional, AGRUPACION).collect()


In [5]:
display(metricas.sample(10))


ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos
i64,i64,str,str,u32,i64,i64,i64
2017,112102,"""Hospital de Ni…","""T019""",2,6,2,0
2005,110140,"""Hospital de Pe…","""N907""",1,1,0,0
2017,112248,"""Hospital Dipre…","""J181""",9,181,1,0
2016,111195,"""Hospital de Ur…","""T855""",1,8,0,0
2009,126100,"""Hospital Clíni…","""Z539""",18,21,1,0
2018,109101,"""Hospital Clíni…","""S680""",2,8,2,0
2017,111195,"""Hospital de Ur…","""K613""",3,48,3,0
2017,119203,"""Hospital Naval…","""H043""",1,8,0,0
2017,109200,"""Hospital Clíni…","""S420""",20,38,19,0


- Al ver una muestra del calculo realizado, se puede observar la cantidad de egresos, dias de
estada, numero de intervenciones quirurgicas y cantidad de muertos para 10 duplas de
hospitales - diagnosticos distintos.

Si bien se tienen las metricas calculadas, ahora se quiere responder a las siguientes preguntas:

- En qué diagnósticos se destaca cada hospital productivamente? Esto significa, en qué diagnósticos
cada hospital tiene una mayor cantidad de egresos que otros hospitales?

- El hospital tiene un desempeño similar dentro de distintos grupos/estratos de hospitales?
El hospital se desempeñara de la misma forma nacionalmente, dentro del estrato público, privado,
etc...?

Para responder a la pregunta se realizará un ranking de todos los hospitales para cada
diagnóstico y cada año observado. Este ranking se realizará para distintos grupos/estratos de
hospitales (Nacional, Hospitales Públicos, Hospitales Privados, Hospitales GRD)

## Análisis de Estratos

In [6]:
dict_estratos = build_features.obtener_diccionario_estratos(
    df_nacional, HOSPITAL_A_ANLIZAR
)

In [7]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales = build_features.agregar_ranking_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

rankings_hospital_analizado = rankings_nacionales.filter(
    pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANLIZAR
)


In [8]:
diags_mas_relevantes = rankings_hospital_analizado.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") > 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)

veinte_mas_relevantes_por_anio = diags_mas_relevantes.groupby(["ANO_EGRESO"]).head(20)

In [9]:
DIAGS_MAS_RELEVANTES = veinte_mas_relevantes_por_anio.select(pl.col("DIAG1").unique()).sort(
    by="DIAG1"
)

In [10]:
diags_mas_relevantes

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2020,112103,"""Instituto Naci…","""Cap.02 NEOPLA…","""C30-C39 NEOPL…","""C34 TUMOR MALI…","""Tumor maligno …","""C341""",76,974,35,8,1,316,0.240506,1,162,0.469136,1,230,0.330435,1,151,0.503311,7,2848,0.026685
2020,112103,"""Instituto Naci…","""Cap.10 ENFERM…","""J80-J84 OTRAS…","""J84 OTRAS ENFE…","""Otras enfermed…","""J841""",58,930,11,6,1,1070,0.054206,1,874,0.066362,1,254,0.228346,1,606,0.09571,8,2848,0.020365
2020,112103,"""Instituto Naci…","""Cap.02 NEOPLA…","""C76-C80 NEOPL…","""C78 TUMOR MALI…","""Tumor maligno …","""C782""",51,556,38,6,1,200,0.255,1,150,0.34,1,101,0.50495,1,143,0.356643,10,2848,0.017907
2020,112103,"""Instituto Naci…","""Cap.09 ENFERM…","""I70-I79 ENFER…","""I71 ANEURISMA …","""Aneurisma de l…","""I712""",29,341,19,0,1,138,0.210145,1,99,0.292929,1,68,0.426471,1,94,0.308511,15,2848,0.010183
2020,112103,"""Instituto Naci…","""Cap.09 ENFERM…","""I70-I79 ENFER…","""I71 ANEURISMA …","""Disección de a…","""I710""",28,530,22,6,1,463,0.060475,1,318,0.08805,1,173,0.16185,1,308,0.090909,16,2848,0.009831
2020,112103,"""Instituto Naci…","""Cap.09 ENFERM…","""I30-I52 OTRAS…","""I35 TRASTORNOS…","""Insuficiencia …","""I351""",27,508,23,1,1,226,0.119469,1,156,0.173077,1,97,0.278351,1,150,0.18,19,2848,0.00948
2020,112103,"""Instituto Naci…","""Cap.10 ENFERM…","""J80-J84 OTRAS…","""J84 OTRAS ENFE…","""Otras enfermed…","""J848""",27,275,6,1,1,107,0.252336,1,76,0.355263,1,58,0.465517,1,71,0.380282,20,2848,0.00948
2020,112103,"""Instituto Naci…","""Cap.19 LESION…","""T80-T88 COMPL…","""T82 COMPLICACI…","""Complicación m…","""T820""",26,475,13,2,1,61,0.42623,1,46,0.565217,1,41,0.634146,1,45,0.577778,22,2848,0.009129
2020,112103,"""Instituto Naci…","""Cap.09 ENFERM…","""I30-I52 OTRAS…","""I47 TAQUICARDI…","""Taquicardia ve…","""I472""",24,150,2,1,1,374,0.064171,1,266,0.090226,1,132,0.181818,1,241,0.099585,24,2848,0.008427


Para hacer un contorl de lo que se calculo, se observará una muestra de 10 hospitales - diagnósticos
distintos

In [11]:
display(rankings_nacionales.sample(10))


ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2014,118107,"""Hospital Clori…","""Cap.18 SÍNTOM…","""R10-R19 SÍNTO…","""R11 NÁUSEA Y V…","""Náusea y vómit…","""R11X""",3,17,0,0,83,1118,0.002683,57,740,0.004054,null,null,null,null,null,null,null,null,null
2012,108204,"""Clínica Río Bl…","""Cap.19 LESION…","""T80-T88 COMPL…","""T83 COMPLICACI…","""Otras complica…","""T838""",2,2,2,0,12,142,0.014085,null,null,null,5,28,0.071429,null,null,null,null,null,null
2001,129100,"""Hospital Dr. M…","""Cap.11 ENFERM…","""K20-K31 ENFER…","""K27 ÚLCERA PÉP…","""Úlcera péptica…","""K275""",2,34,0,0,6,44,0.045455,6,44,0.045455,null,null,null,6,40,0.05,null,null,null
2012,109201,"""Clínica Dávila…","""Cap.11 ENFERM…","""K00-K14 ENFER…","""K11 ENFERMEDAD…","""Mucocele de gl…","""K116""",3,3,3,0,6,74,0.040541,null,null,null,4,45,0.066667,null,null,null,null,null,null
2009,108100,"""Hospital de Sa…","""Cap.18 SÍNTOM…","""R50-R69 SÍNTO…","""R57 CHOQUE, NO…","""Otras formas d…","""R578""",1,1,0,1,7,27,0.037037,5,18,0.055556,null,null,null,5,18,0.055556,null,null,null
2019,106100,"""Hospital Carlo…","""Cap.08 ENFERM…","""H80-H83 ENFER…","""H81 TRASTORNOS…","""Vértigo paroxí…","""H811""",1,3,0,0,88,167,0.005988,51,80,0.0125,null,null,null,27,46,0.021739,null,null,null
2001,121202,"""Clínica Aleman…","""Cap.11 ENFERM…","""K00-K14 ENFER…","""K07 ANOMALÍAS …","""Anomalías evid…","""K070""",3,14,3,0,4,70,0.042857,null,null,null,3,45,0.066667,null,null,null,null,null,null
2012,107100,"""Hospital Dr. G…","""Cap.19 LESION…","""S00-S09 TRAUM…","""S01 HERIDA DE …","""Herida de otra…","""S018""",17,40,10,0,13,1021,0.01665,10,634,0.026814,null,null,null,10,508,0.033465,null,null,null
2019,118202,"""Clínica de la …","""Cap.10 ENFERM…","""J90-J94 OTRAS…","""J93 NEUMOTÓRAX…","""Neumotórax esp…","""J930""",2,7,0,0,23,138,0.014493,null,null,null,11,59,0.033898,null,null,null,null,null,null


En la muestra anterior se puede observar las métricas previamente calculada, y el ranking
respectivo que tiene cada hospital para diagnósticos en cada año específico.

In [12]:
rankings_nacionales.to_pandas().to_csv(
    "../data/interim/ranking_nacional_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

rankings_hospital_analizado.to_pandas().to_csv(
    "../data/interim/ranking_torax_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.

In [13]:
df_nacional_mayores_a_14 = df_nacional.filter(pl.col("EDAD_A_OS") > 14)

In [14]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(
    df_nacional_mayores_a_14, AGRUPACION
).collect()


In [15]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales_sobre_14 = build_features.agregar_ranking_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

diags_congenitos_relevantes = rankings_nacionales_sobre_14.filter(
    (pl.col("DIAG1").is_in(["Q230", "Q211", "Q231"])) & (
        pl.col("ANO_EGRESO") == 2019)
)


In [16]:
diags_congenitos_relevantes.to_pandas().to_csv(
    "../data/interim/diags_congenitas_sobre_14_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## 3. Análisis de procedimientos DEIS

En este análisis se quieren responder las siguientes preguntas:

1. ¿Cuál es el procedimiento más frecuente dentro de los pacientes que se les realizó un procedimiento
quirúrgico?

In [41]:
proced = (
    df_nacional.filter(pl.col("INTERV_Q") == 1)
    .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])
    .agg(pl.count())
).collect()

In [46]:
display(proced)

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_PROCED_PPAL,count
i64,i64,str,u32
2001,110100,null,5294
2001,101213,null,1014
2001,114102,null,379
2001,109104,null,2
2001,113150,null,1711
2001,115104,null,1022
2001,110150,null,2478
2001,124120,null,410
2001,124125,null,401


Los resultados muestran el desglose por año y recinto hospitalario junto a la cantidad de
procedimientos realizados en cada intervención quirúrgica. Sin embargo, se puede observar que
existe una cantidad importante de casos donde la glosa del procedimiento fue nula.

Ahora, se quiere calcular cuántas intervenciones quirúrgicas carecen de una glosa de procedimiento.

In [55]:
display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))

GLOSA_PROCED_PPAL,count
str,u32
"""Histerosalping…",1
"""Respuesta resp…",1
"""Punción evacua…",1
"""Radiografía re…",1
"""Sinusoscopia d…",1
"""Cintigrafía re…",1
"""Perfil uretral…",1
"""Infiltraciones…",1
"""Ecobiometria c…",1


Por lo tanto, 12382642 de todas las intervenciones quirúrgicas carecen de una glosa de procedimiento.

In [57]:
proced.write_excel("../data/interim/conteo_procedimientos.xlsx")